In [1]:
import numpy as np
import pandas as pd
import random
import os
from os.path import expanduser
import sys
from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_context('poster')

code_base='./'
sys.path.insert(0, code_base)
sys.path.insert(0, code_base+'sampler')
sys.path.insert(0, code_base+'models')

from utility import data_transformation, transform_with_keys, generate_mixed_events
from utility import get_entity_samplers_and_noise_prob, get_entity_type_sampler_and_mappings

from sklearn.metrics import average_precision_score, roc_auc_score
from metrics_ranking import eval_multiple, eval_apk

import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Layer, Input, Dense, Embedding, Flatten, Merge, AveragePooling1D, Merge, Permute, merge
from keras.regularizers import WeightRegularizer, l1, l2, activity_l2, activity_l1

/Users/akdidier/virtualenvironments/APE-keras/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/Users/akdidier/virtualenvironments/APE-keras/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/Users/akdidier/virtualenvironments/APE-keras/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/Users/akdidier/virtualenvironments/APE-keras/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import ha

# Data and utility preparations

In [2]:
# data locations
data_folder = 'demo_toy/'                      # it only contains unrunnable toy data for demonstration
data_event_file = data_folder + '/events.csv'  # historical events without any label
data_test_file = data_folder + '/test.csv'     # test (future) events with additional label column

In [3]:
# load data
home = expanduser('~')
table = pd.read_csv(data_event_file)
table_test = pd.read_csv(data_test_file)

# index the entities in data
table_transformed, id2type_and_name, type_and_name2id, type2range = data_transformation(table)
table_transformed_test = transform_with_keys(table_test, table_test.columns[:-1], type_and_name2id)
# drop rows in test with NaN if there are any (imputation can also be used here)
table_transformed_test = table_transformed_test.dropna()

# sampler preparation
type2sampler, noise_prob = get_entity_samplers_and_noise_prob(table_transformed, noise_prob_cal='logkPn@10', neg_dist='unigram')
type2typeid, typeid2type, entity_type_sample, type_cad_dist = get_entity_type_sampler_and_mappings(table_transformed, neg_dist='uniform')

NameError: global name 'MultinomialSampler' is not defined

In [4]:
table.head()

,hour_of_day,which_day,uid,proc_src_name,proc_dst_name,proc_src_first3_folder_path,proc_dst_first3_folder_path
0,hour_of_day.0,which_day.6,uid.68,proc_src_name.l.lf,proc_dst_name.l.jj,proc_src_first3_folder_path./usr/sbin/,proc_dst_first3_folder_path./usr/lib/
1,hour_of_day.0,which_day.6,uid.0,proc_src_name.l.bash,proc_dst_name.l.s,proc_src_first3_folder_path./bin/,proc_dst_first3_folder_path./usr/sbin/


In [5]:
table_transformed

,hour_of_day,which_day,uid,proc_src_name,proc_dst_name,proc_src_first3_folder_path,proc_dst_first3_folder_path
0,0,1,2,4,6,9,11
1,0,1,3,5,7,8,10


In [6]:
id2type_and_name

{0: ('hour_of_day', 'hour_of_day.0'),
 1: ('which_day', 'which_day.6'),
 2: ('uid', 'uid.68'),
 3: ('uid', 'uid.0'),
 4: ('proc_src_name', 'proc_src_name.l.lf'),
 5: ('proc_src_name', 'proc_src_name.l.bash'),
 6: ('proc_dst_name', 'proc_dst_name.l.jj'),
 7: ('proc_dst_name', 'proc_dst_name.l.s'),
 8: ('proc_src_first3_folder_path', 'proc_src_first3_folder_path./bin/'),
 9: ('proc_src_first3_folder_path', 'proc_src_first3_folder_path./usr/sbin/'),
 10: ('proc_dst_first3_folder_path', 'proc_dst_first3_folder_path./usr/sbin/'),
 11: ('proc_dst_first3_folder_path', 'proc_dst_first3_folder_path./usr/lib/')}

In [7]:
type_and_name2id

{('hour_of_day', 'hour_of_day.0'): 0,
 ('proc_dst_first3_folder_path', 'proc_dst_first3_folder_path./usr/lib/'): 11,
 ('proc_dst_first3_folder_path', 'proc_dst_first3_folder_path./usr/sbin/'): 10,
 ('proc_dst_name', 'proc_dst_name.l.jj'): 6,
 ('proc_dst_name', 'proc_dst_name.l.s'): 7,
 ('proc_src_first3_folder_path', 'proc_src_first3_folder_path./bin/'): 8,
 ('proc_src_first3_folder_path', 'proc_src_first3_folder_path./usr/sbin/'): 9,
 ('proc_src_name', 'proc_src_name.l.bash'): 5,
 ('proc_src_name', 'proc_src_name.l.lf'): 4,
 ('uid', 'uid.0'): 3,
 ('uid', 'uid.68'): 2,
 ('which_day', 'which_day.6'): 1}

In [8]:
type2range

{'hour_of_day': [0, 0],
 'proc_dst_first3_folder_path': [10, 11],
 'proc_dst_name': [6, 7],
 'proc_src_first3_folder_path': [8, 9],
 'proc_src_name': [4, 5],
 'uid': [2, 3],
 'which_day': [1, 1]}

In [4]:
table_transformed_test

,hour_of_day,which_day,uid,proc_src_name,proc_dst_name,proc_src_first3_folder_path,proc_dst_first3_folder_path,label
0,0,1,2,4,6,9,11,1
1,0,1,3,5,7,8,10,0


In [10]:
type2sampler

{'hour_of_day': <function sample_batch>,
 'proc_dst_first3_folder_path': <function sample_batch>,
 'proc_dst_name': <function sample_batch>,
 'proc_src_first3_folder_path': <function sample_batch>,
 'proc_src_name': <function sample_batch>,
 'uid': <function sample_batch>,
 'which_day': <function sample_batch>}

In [11]:
noise_prob

array([2.3025851, 2.3025851, 1.609438 , 1.609438 , 1.609438 , 1.609438 ,
       1.609438 , 1.609438 , 1.609438 , 1.609438 , 1.609438 , 1.609438 ],
      dtype=float32)

In [12]:
entity_type_sample

<function sample_batch>

In [13]:
type_cad_dist

array([1., 1., 1., 1., 1., 1., 1.])

In [5]:
class DataSpec(object):
    def __init__(self, table_transformed, type2range):
        self.num_entity_type = len(table_transformed.columns)
        self.num_entity = max([max(type_range) for each, type_range in type2range.iteritems()]) - \
                          min([min(type_range) for each, type_range in type2range.iteritems()]) + 1
            
data_spec = DataSpec(table_transformed, type2range)

# Model specifications and run

In [6]:
class Conf(object):
    def __init__(self):
        self.max_epoch = 10
#         self.batch_size = 512
        self.batch_size = 2
        self.num_negatives = 5
        self.emb_dim = 3
        self.loss = 'skip-gram'
        #self.loss = 'max-margin'
        self.no_weight = False
        self.ignore_noise_dist = False
        
conf = Conf()
import ape
reload(ape)
from ape import get_model
model = get_model(conf, data_spec)

entity emb
Tensor("Gather:0", shape=(?, 7, 3), dtype=float32)
entity_emb_t
Tensor("transpose:0", shape=(?, 3, 7), dtype=float32)


In [8]:
# main logics

abandon_uneven_batch = False
batch_size = conf.batch_size
num_negatives = conf.num_negatives
events = np.array(table_transformed)
num_iters = np.ceil(events.shape[0] / float(batch_size)).astype(int)
for epoch in range(1, conf.max_epoch + 1):
    np.random.shuffle(events)
    cost = 0
    entity_type_assigns = entity_type_sample(num_iters)

    for it in range(num_iters):
        neg_entity_typeid = entity_type_assigns[it]
        events_batch = events[it * batch_size: (it + 1) * batch_size]
        if abandon_uneven_batch and events_batch.shape[0] != batch_size:
            continue
            
       
        #Why is events noise_prob the same scalar?
        events_batch_mixed, events_noise_prob, events_label = \
            generate_mixed_events(events_batch, neg_entity_typeid, num_negatives, type2sampler, typeid2type, noise_prob)
        
        print 'events_batch_mixed '
        print events_batch_mixed
#         print events_noise_prob
        print 'events_label'
        print events_label
#         print 'input looks like '
        inp = model.input
        outputs = [layer.output for layer in model.layers]
        functors = [K.function([inp], [out]) for out in outputs]
        print 'input: '
        print [events_batch_mixed[0:2]]
        layer_outs = [func([events_batch_mixed[0:2]]) for func in functors]
        
        for i in range(len(outputs)):
            print outputs[i]
            print layer_outs[i]
        #         print layer_outs
#         print 'output per layer looks like'
#         for output in outputs:
#             print output
#         cost += model.train_on_batch([events_batch_mixed, events_noise_prob], events_label)
#     break
#     print '[INFO] epoch %d, cost: %f' % (epoch, cost), 'norm', np.sqrt(np.mean(model.get_weights()[0]**2))

inside generate_mixed_events
events_batch_mixed 
[[ 0  1  2  4  6  9 11]
 [ 0  1  3  5  7  8 10]
 [ 0  1  3  4  6  9 11]
 [ 0  1  3  4  6  9 11]
 [ 0  1  3  4  6  9 11]
 [ 0  1  3  4  6  9 11]
 [ 0  1  3  4  6  9 11]
 [ 0  1  2  5  7  8 10]
 [ 0  1  2  5  7  8 10]
 [ 0  1  2  5  7  8 10]
 [ 0  1  2  5  7  8 10]
 [ 0  1  3  5  7  8 10]]
events_label
[ 1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
input: 
[array([[ 0,  1,  2,  4,  6,  9, 11],
       [ 0,  1,  3,  5,  7,  8, 10]])]
Tensor("input_1:0", shape=(?, 7), dtype=int32)
[array([[ 0,  1,  2,  4,  6,  9, 11],
       [ 0,  1,  3,  5,  7,  8, 10]], dtype=int32)]
Tensor("Gather:0", shape=(?, 7, 3), dtype=float32)
[array([[[ 0.00958629, -0.04908265, -0.01534686],
        [-0.04556859, -0.01202967, -0.04494147],
        [-0.02479097,  0.00463505, -0.02525231],
        [ 0.00020622, -0.03255018,  0.04221297],
        [ 0.02903252,  0.00798158,  0.03300362],
        [ 0.00910008,  0.00250788, -0.02446922],
        [ 0.02940017,  0.038092

Tensor("input_1:0", shape=(?, 7), dtype=int32)
[array([[ 0,  1,  2,  4,  6,  9, 11],
       [ 0,  1,  3,  5,  7,  8, 10]], dtype=int32)]
Tensor("Gather:0", shape=(?, 7, 3), dtype=float32)
[array([[[ 0.00958629, -0.04908265, -0.01534686],
        [-0.04556859, -0.01202967, -0.04494147],
        [-0.02479097,  0.00463505, -0.02525231],
        [ 0.00020622, -0.03255018,  0.04221297],
        [ 0.02903252,  0.00798158,  0.03300362],
        [ 0.00910008,  0.00250788, -0.02446922],
        [ 0.02940017,  0.0380922 , -0.03365039]],

       [[ 0.00958629, -0.04908265, -0.01534686],
        [-0.04556859, -0.01202967, -0.04494147],
        [-0.00133361,  0.01887629, -0.02472365],
        [ 0.02528331,  0.01508728,  0.04826378],
        [ 0.03219731, -0.01187993, -0.03705746],
        [-0.02257785,  0.04708144, -0.04200212],
        [ 0.01719493, -0.04235654,  0.00732162]]], dtype=float32)]
Tensor("transpose:0", shape=(?, 3, 7), dtype=float32)
[array([[[ 0.00958629, -0.04556859, -0.02479097,  0

In [41]:
events

array([[ 0,  1,  2,  4,  6,  9, 11],
       [ 0,  1,  3,  5,  7,  8, 10]])

In [42]:
num_iters

1

In [53]:
-0.00328965*-0.00328965

1.0821797122499999e-05

# Evaluation

In [17]:
# evaluation system anomaly
def eval_print(table_transformed_w_label):
    truth = np.array(table_transformed_w_label)[:, -1]
    truth[truth > 0] = 1
    truth = truth* -1 + 1
    print 'truth_mean', np.mean(truth)
    pred = -model.predict([np.array(table_transformed_w_label)[:, :-1], np.zeros(table_transformed_w_label.shape[0])], batch_size=1024)[:, 0]
    perturbation = np.random.random(pred.shape) * 1e-4
    pred += perturbation
    pred_random = pred.copy()
    np.random.shuffle(pred_random)
    print 'pred_mean', np.mean(pred)
    print 'AP', average_precision_score(truth, pred)#, average_precision_score(truth, pred_random)
    print 'AUC', roc_auc_score(truth, pred)#, roc_auc_score(truth, pred_random)
    
eval_print(table_transformed_test)

truth_mean 0.5
pred_mean 0.011138252
AP 0.5
AUC 0.0
